In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import Table, func
from dateutil.relativedelta import relativedelta
from datetime import datetime
from flask_mail import Mail, Message
from Models.helper import u_id_generator, f_id_generator, r_id_generator
# from Models.model import User, Film, Review, Review_Like, Review_Dislike
from datetime import datetime
import bcrypt

In [2]:
app = Flask(__name__)

db_config = {
    'SQLALCHEMY_DATABASE_URI' : 'sqlite:///Database/doubi_database.db',
    'SQLALCHEMY_TRACK_MODIFICATIONS' : False
}

app.config.update(db_config)
db = SQLAlchemy(app)

In [3]:
followers = Table('followers', db.metadata,
    db.Column('followed_id', db.String(32), db.ForeignKey('user.u_id')),
    db.Column('follower_id', db.String(32), db.ForeignKey('user.u_id'))
)

blocked_users = Table('blocked_users', db.metadata,
    db.Column('blocked_id', db.String(32), db.ForeignKey('user.u_id')),
    db.Column('blocker_id', db.String(32), db.ForeignKey('user.u_id')),
)

users_wish_film = Table('users_wish_film', db.metadata,
    db.Column('user_id', db.String(32), db.ForeignKey('user.u_id')),
    db.Column('film_id', db.String(32), db.ForeignKey('film.f_id'))
)


class User(db.Model):
    __tablename__ = 'user'
    
    u_id = db.Column(db.String(32), primary_key=True, nullable=False, unique=True, default=u_id_generator)
    username = db.Column(db.String(80), nullable=False, unique=True)
    password_hash = db.Column(db.Text, nullable=False)
    email = db.Column(db.String(80), nullable=False, unique=True)
    url_photo = db.Column(db.Text, nullable=True)
    is_admin = db.Column(db.Boolean, nullable=False, default=False)
    is_blocked = db.Column(db.Boolean, nullable=False, default=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    updated_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow, onupdate=datetime.utcnow)

    followed = db.relationship('User', 
                                secondary=followers,
                                primaryjoin=(followers.c.follower_id == u_id),
                                secondaryjoin=(followers.c.followed_id == u_id),
                                backref=db.backref('followers', lazy='dynamic'),
                                lazy='dynamic')

    blocked = db.relationship('User',
                                secondary=blocked_users,
                                primaryjoin=(blocked_users.c.blocker_id == u_id),
                                secondaryjoin=(blocked_users.c.blocked_id == u_id),
                                backref=db.backref('blockers', lazy='dynamic'),
                                lazy='dynamic')
    
    reviews = db.relationship('Review', backref='user', lazy='dynamic')

    review_likes = db.relationship('Review_Like', backref='user', lazy='dynamic')
    review_dislikes = db.relationship('Review_Dislike', backref='user', lazy='dynamic')
    
    wish = db.relationship('Film', secondary=users_wish_film, backref='user', lazy='dynamic')
    
    @property
    def password(self):
        raise AttributeError('password is not a readable attribute')

    @password.setter
    def password(self, password):
        salt = bcrypt.gensalt()
        self.password_hash = bcrypt.hashpw(password.encode('utf-8'), salt)
    
    def verify_password(self, password):
        return bcrypt.checkpw(password.encode('utf-8'), self.password_hash)


    

class Film(db.Model):
    __tablename__ = 'film'

    f_id = db.Column(db.String(32), primary_key=True, nullable=False, unique=True, default=f_id_generator)
    title = db.Column(db.String(80), nullable=False)
    genre = db.Column(db.String(80), nullable=False)
    year = db.Column(db.Integer, nullable=True)
    run_time = db.Column(db.String(16), nullable=True)
    rating_imdb = db.Column(db.Float, nullable=True)
    overview = db.Column(db.String(500), nullable=True)
    director = db.Column(db.String(80), nullable=True)
    actor = db.Column(db.String(200), nullable=True)
    url_poster = db.Column(db.Text, nullable=True)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    updated_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow, onupdate=datetime.utcnow)

    reviews = db.relationship('Review', backref='film', lazy='dynamic')
    
    wish_by = db.relationship('User', secondary=users_wish_film, backref='film', lazy='dynamic')
    
    


class Review(db.Model):
    __tablename__ = 'review'

    r_id = db.Column(db.String(32), primary_key=True, nullable=False, unique=True, default=r_id_generator)
    u_id = db.Column(db.String(32), db.ForeignKey('user.u_id'), nullable=False)
    f_id = db.Column(db.String(32), db.ForeignKey('film.f_id'), nullable=False)
    content = db.Column(db.String(500), nullable=True)
    rating = db.Column(db.Integer, nullable=False)
    bad_word = db.Column(db.Boolean, nullable=False, default=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

    likes = db.relationship('Review_Like', backref='review', lazy='dynamic')
    dislikes = db.relationship('Review_Dislike', backref='review', lazy='dynamic')


class Review_Like(db.Model):
    __tablename__ = 'review_like'

    r_id = db.Column(db.String(32), db.ForeignKey('review.r_id'), primary_key=True, nullable=False)
    u_id = db.Column(db.String(32), db.ForeignKey('user.u_id'), primary_key=True, nullable=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)


class Review_Dislike(db.Model):
    __tablename__ = 'review_dislike'

    r_id = db.Column(db.String(32), db.ForeignKey('review.r_id'), primary_key=True, nullable=False)
    u_id = db.Column(db.String(32), db.ForeignKey('user.u_id'), primary_key=True, nullable=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

In [4]:
# from sqlite3 import connect
# conn = connect(app.root_path + '/Database/doubi_database_old.db')
# c = conn.cursor()

# c.execute('SELECT * FROM film')
# films = c.fetchall()
# print(films)
db.create_all()

In [5]:
from sqlite3 import connect
conn = connect(app.root_path + '/Database/doubi_database_old.db')
c = conn.cursor()

c.execute('SELECT * FROM film')
films = c.fetchall()
# films

In [6]:
db.session.rollback()
for film in films:
    f = Film(
        f_id = film[0],
        title = film[1],
        genre = film[2],
        year = film[3],
        run_time = film[4],
        rating_imdb = film[5],
        overview = film[6],
        director = film[7],
        actor = film[8],
        url_poster = film[9],

        # convert string to datetime
        created_time = datetime.strptime(film[10], '%Y-%m-%d %H:%M:%S.%f'),
        updated_time = datetime.strptime(film[11], '%Y-%m-%d %H:%M:%S.%f')
        )
    db.session.add(f)
db.session.commit()
        

C:\Users\zzy80\AppData\Local\Temp\ipykernel_19876\3107100934.py:3: SAWarning: relationship 'User.film' will copy column user.u_id to column users_wish_film.user_id, which conflicts with relationship(s): 'Film.user' (copies user.u_id to users_wish_film.user_id), 'User.wish' (copies user.u_id to users_wish_film.user_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="user,wish"' to the 'User.film' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
  f = Film(
C:\Users\zzy80\AppData\Local\Temp\ipykernel_19876\3107100934.py:3: SAWarning: relationship 'User.film' will copy column film.f_id to column users_wish_f

In [7]:
names = []
for film in films:
    name = film[7].split()
    names += name
    
    name = film[8].split(',')
    for n in name:
        names += n.split()

# remove duplicates
names = list(set(names))

# remove names less than 3 characters
names = [n for n in names if len(n) > 2]

import random
# select 1000 random names
names = random.sample(names, 1000)
# print(names)

In [8]:
db.session.rollback()
count = 0
for name in names:
    u = User(
        username = name,
        password = name[0:3] + '_pwd123',
        email = name + '@gmail.com'
    )
    db.session.add(u)
    count += 1
    print("{} ({}/1000) is added".format(name, count))
    
db.session.commit()

Aniruddha (1/1000) is added
Weisz (2/1000) is added
Gurira (3/1000) is added
Boman (4/1000) is added
Merlant (5/1000) is added
Bernhard (6/1000) is added
Mia (7/1000) is added
Julie (8/1000) is added
Bauer (9/1000) is added
Lina (10/1000) is added
Dormer (11/1000) is added
Swinton (12/1000) is added
Irani (13/1000) is added
Prateik (14/1000) is added
Poppe (15/1000) is added
Dougnac (16/1000) is added
Squires (17/1000) is added
Solveig (18/1000) is added
Campbell (19/1000) is added
Jory (20/1000) is added
Chong (21/1000) is added
Kishimoto (22/1000) is added
Inoue (23/1000) is added
Rex (24/1000) is added
Stiller (25/1000) is added
Tilda (26/1000) is added
Iko (27/1000) is added
Anand (28/1000) is added
Yeoh (29/1000) is added
Aldo (30/1000) is added
Cuarón (31/1000) is added
Angus (32/1000) is added
Addy (33/1000) is added
Godard (34/1000) is added
Pitt (35/1000) is added
Shimamoto (36/1000) is added
Srinath (37/1000) is added
Geoffrey (38/1000) is added
Jonze (39/1000) is added
Farid

In [13]:
pwd = '123456'

# encode password
pwd_hash = bcrypt.hashpw(pwd.encode('utf-8'), bcrypt.gensalt())
print(pwd_hash)

b'$2b$12$YXrl80ioPqupOesc3AtusewUoceI9ib3m5Zb6hQ6JCxBWINPfXsHG'


In [14]:
bcrypt.checkpw(pwd.encode('utf-8'), pwd_hash)

True